importing the basic libraries for data analysis 

In [12]:
import pandas as pd
import numpy as np
import matplotlib as plt 

importing the excel files to  python as dataframes 


In [17]:
internal = pd.read_excel('internal_trade_data.xlsx')
clearer = pd.read_excel('clearer_trade_data.xlsx')


seeing the data to validate that it got imported and to get some knowledge about the data 

In [19]:
internal.head()


,trade_id,contract_type,contract_period,quantity_mwh,price_eur_per_mwh
0,1,Outright,2025-03,100,30
1,2,Outright,2025-04,200,31
2,3,Outright,2025-05,250,32
3,4,Outright Quarter,Q225,300,33


In [20]:
internal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   trade_id           4 non-null      int64 
 1   contract_type      4 non-null      object
 2   contract_period    4 non-null      object
 3   quantity_mwh       4 non-null      int64 
 4   price_eur_per_mwh  4 non-null      int64 
dtypes: int64(3), object(2)
memory usage: 292.0+ bytes


In [21]:
clearer.head()

,delivery_month,quantity_mwh,price_eur_per_mwh
0,2025-03,100,30.0
1,2025-04,500,32.0
2,2025-05,550,32.5
3,2025-06,300,33.0


In [22]:
clearer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   delivery_month     4 non-null      object 
 1   quantity_mwh       4 non-null      int64  
 2   price_eur_per_mwh  4 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 228.0+ bytes


extracting the quarterly contract from the internal file sheet to represent it as monthely legs

In [23]:

quarterly = internal[internal['contract_period'] == 'Q225'].copy()


months_in_quarter = 3


monthly_legs = pd.DataFrame({
    'trade_id': ['Q225-Apr', 'Q225-May', 'Q225-Jun'],
    'contract_type': ['Quarter'] * 3,
    'contract_period': ['2025-04', '2025-05', '2025-06'],
    'quantity_mwh': quarterly['quantity_mwh'].values[0] / months_in_quarter,
    'price_eur_per_mwh': quarterly['price_eur_per_mwh'].values[0]
})


getting an overview of the monthly_legs dataframes and verfying that the sum of the monthely legs is the same as the quartely

In [24]:

total_quantity = monthly_legs['quantity_mwh'].sum()
print(f"Total quantity after splitting: {total_quantity}")

Total quantity after splitting: 300.0


In [26]:
average_price = monthly_legs['price_eur_per_mwh'].mean()
print(f"Average price after splitting: {average_price}")

Average price after splitting: 33.0


In [27]:
original_quantity = quarterly['quantity_mwh'].values[0]
original_price = quarterly['price_eur_per_mwh'].values[0]
original_value = original_quantity * original_price

split_value = (monthly_legs['quantity_mwh'] * monthly_legs['price_eur_per_mwh']).sum()

print(f"Original contract value: {original_value}")
print(f"Sum of split contract values: {split_value}")

Original contract value: 9900
Sum of split contract values: 9900.0


In [25]:
monthly_legs

,trade_id,contract_type,contract_period,quantity_mwh,price_eur_per_mwh
0,Q225-Apr,Quarter,2025-04,100.0,33
1,Q225-May,Quarter,2025-05,100.0,33
2,Q225-Jun,Quarter,2025-06,100.0,33


after verfying i have made sure that the sum of the monthly really corresponds to the quarterly which is 100.0 @ 33 and they have the same
total of 9900.0

now i will extract the monthly contracts from the internal file

In [28]:
outright = internal[internal['contract_type'] == 'Outright'].copy()
outright

,trade_id,contract_type,contract_period,quantity_mwh,price_eur_per_mwh
0,1,Outright,2025-03,100,30
1,2,Outright,2025-04,200,31
2,3,Outright,2025-05,250,32


Renaming the  contract_period to delivery_month for easier merging later. doing this to match the names of the outright and monthly legs to each other

In [29]:
outright = outright.rename(columns={'contract_period': 'delivery_month'})
monthly_legs = monthly_legs.rename(columns={'contract_period': 'delivery_month'})
outright


,trade_id,contract_type,delivery_month,quantity_mwh,price_eur_per_mwh
0,1,Outright,2025-03,100,30
1,2,Outright,2025-04,200,31
2,3,Outright,2025-05,250,32


In [33]:
monthly_legs

,trade_id,contract_type,delivery_month,quantity_mwh,price_eur_per_mwh
0,Q225-Apr,Quarter,2025-04,100.0,33
1,Q225-May,Quarter,2025-05,100.0,33
2,Q225-Jun,Quarter,2025-06,100.0,33


Combine both DataFrames vertically

In [31]:

combined = pd.concat([
    outright[['delivery_month', 'quantity_mwh', 'price_eur_per_mwh']],
    monthly_legs[['delivery_month', 'quantity_mwh', 'price_eur_per_mwh']]
])
combined

,delivery_month,quantity_mwh,price_eur_per_mwh
0,2025-03,100.0,30
1,2025-04,200.0,31
2,2025-05,250.0,32
0,2025-04,100.0,33
1,2025-05,100.0,33
2,2025-06,100.0,33


creating a function to get the weighted avg price 

In [34]:
def weighted_avg_price(df):
    return (df['quantity_mwh'] * df['price_eur_per_mwh']).sum() / df['quantity_mwh'].sum()

In [37]:
net_internal = (
    combined.groupby('delivery_month')
    .apply(
        lambda x: pd.Series({
            'quantity_mwh': x['quantity_mwh'].sum(),
            'price_eur_per_mwh': (
                (x['quantity_mwh'] * x['price_eur_per_mwh']).sum() / x['quantity_mwh'].sum()
            )
        }),
        include_groups=False  # This silences the warning
    )
    .reset_index()
)


In [38]:
net_internal

,delivery_month,quantity_mwh,price_eur_per_mwh
0,2025-03,100.0,30.000000
1,2025-04,300.0,31.666667
2,2025-05,350.0,32.285714
3,2025-06,100.0,33.000000


Merge the net internal data with clearer data on delivery_month

In [39]:
reconciliation = pd.merge(net_internal, clearer, on='delivery_month', suffixes=('_internal', '_clearer'))

Calculate differences to spot mismatches

In [40]:

reconciliation['quantity_diff'] = reconciliation['quantity_mwh_internal'] - reconciliation['quantity_mwh_clearer']
reconciliation['price_diff'] = reconciliation['price_eur_per_mwh_internal'] - reconciliation['price_eur_per_mwh_clearer']

reconciliation

,delivery_month,quantity_mwh_internal,price_eur_per_mwh_internal,quantity_mwh_clearer,price_eur_per_mwh_clearer,quantity_diff,price_diff
0,2025-03,100.0,30.000000,100,30.0,0.0,0.000000
1,2025-04,300.0,31.666667,500,32.0,-200.0,-0.333333
2,2025-05,350.0,32.285714,550,32.5,-200.0,-0.214286
3,2025-06,100.0,33.000000,300,33.0,-200.0,0.000000


After netting internal trades and decomposing the quarterly contract (Q225), we compare internal and clearer data.

The internal data includes both outright monthly trades and a quarterly contract (Q2 2025), which was evenly split across April, May, and June.
We calculated the total quantity and weighted average price for each month based on internal data.
Then, we compared it to the clearer’s official monthly data.

     Final Reconciliation Output

Month	Quantity Match	      Price Match
March	✅ Match	         ✅ Match
April	❌ 200 MWh missing	 ❌ Slight price mismatch
May	    ❌ 200 MWh missing	 ❌ Slight price mismatch
June	❌ 200 MWh missing	 ✅ Match
